In [34]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
from hyperopt import STATUS_OK, Trials, hp, space_eval, tpe, fmin
import lightgbm as lgb
from matplotlib import pyplot as plt
from matplotlib import rcParams
import numpy as np
from pathlib import Path
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold, train_test_split
import seaborn as sns
import warnings
rcParams['figure.figsize'] = (16, 8)
plt.style.use('fivethirtyeight')
pd.set_option('max_columns', 100)
pd.set_option("display.precision", 4)
warnings.simplefilter('ignore')

data_dir = Path('C:\\Users\\DKU\\Desktop\\단국대\\')
feature_dir = Path('C:\\Users\\DKU\\Desktop\\단국대\\build\\fea\\')
val_dir = Path('C:\\Users\\DKU\\Desktop\\단국대\\build\\val\\')
tst_dir = Path('C:\\Users\\DKU\\Desktop\\단국대\\build\\tst\\')
sub_dir = Path('C:\\Users\\DKU\\Desktop\\단국대\\build\\sub\\')

trn_file = data_dir / 'train.csv'
tst_file = data_dir / 'test.csv'
sample_file = data_dir / 'sample_submission.csv'

target_col = 'class'
n_fold = 5
n_class = 3
seed = 2022

algo_name = 'lgb_hyperopt'
feature_name = 'feature'
model_name = f'{algo_name}_{feature_name}'

feature_file = feature_dir / f'{feature_name}.csv'
p_val_file = val_dir / f'{model_name}.val.csv'
p_tst_file = tst_dir / f'{model_name}.tst.csv'
sub_file = sub_dir / f'{model_name}.csv'
df = pd.read_csv(feature_file, index_col=0)


In [35]:
y = df[target_col].values[:320000]
df.drop(target_col, axis=1, inplace=True)
trn = df.iloc[:320000].values
tst = df.iloc[320000:].values
feature_name = df.columns.tolist()
print(y.shape, trn.shape, tst.shape)

(320000,) (320000, 19) (80000, 19)


In [36]:
X_trn, X_val, y_trn, y_val = train_test_split(trn, y, test_size=.2, random_state=seed)


In [37]:
params = {
    "objective": "multiclass",
    "n_estimators": 1000,
    "subsample_freq": 1,
    "random_state": seed,
    "n_jobs": -1,
}

space = {
    "learning_rate": hp.loguniform("learning_rate", np.log(0.01), np.log(0.3)),
    "num_leaves": hp.choice("num_leaves", [15, 31, 63, 127]),
    "colsample_bytree": hp.quniform("colsample_bytree", .5, .9, 0.1),
    "subsample": hp.quniform("subsample", .5, .9, 0.1),
    "min_child_samples": hp.choice('min_child_samples', [10, 25, 100])
}

def objective(hyperparams):
    model = lgb.LGBMClassifier(**params, **hyperparams)
    model.fit(X=X_trn, y=y_trn,
              eval_set=[(X_val, y_val)],
              eval_metric="multi_logloss",
              early_stopping_rounds=10,
              verbose=False)
    score = model.best_score_["valid_0"]["multi_logloss"]

    return {'loss': score, 'status': STATUS_OK, 'model': model}

trials = Trials()
best = fmin(fn=objective, space=space, trials=trials,
            algo=tpe.suggest, max_evals=10, verbose=1)

hyperparams = space_eval(space, best)
n_best = trials.best_trial['result']['model'].best_iteration_
params.update(hyperparams)
print(params)

100%|████████████████████████████████████████████████████████████████████████████████████| 10/10 [03:56<00:00, 23.64s/trial, best loss: 0.16136988210468498]
{'objective': 'multiclass', 'n_estimators': 1000, 'subsample_freq': 1, 'random_state': 2022, 'n_jobs': -1, 'colsample_bytree': 0.5, 'learning_rate': 0.014151324855207464, 'min_child_samples': 10, 'num_leaves': 127, 'subsample': 0.9}


In [38]:
cv = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed)


In [39]:
p_val = np.zeros((trn.shape[0], n_class))
p_tst = np.zeros((tst.shape[0], n_class))
for i, (i_trn, i_val) in enumerate(cv.split(trn, y), 1):
    print(f'training model for CV #{i}')
    clf = lgb.LGBMClassifier(**params)
    clf.fit(trn[i_trn], y[i_trn],
            eval_set=[(trn[i_val], y[i_val])],
            eval_metric='multiclass',
            early_stopping_rounds=10)
    
    p_val[i_val, :] = clf.predict_proba(trn[i_val])
    p_tst += clf.predict_proba(tst) / n_fold

training model for CV #1
[1]	valid_0's multi_logloss: 0.968052
Training until validation scores don't improve for 10 rounds
[2]	valid_0's multi_logloss: 0.950415
[3]	valid_0's multi_logloss: 0.932636
[4]	valid_0's multi_logloss: 0.915425
[5]	valid_0's multi_logloss: 0.899326
[6]	valid_0's multi_logloss: 0.883348
[7]	valid_0's multi_logloss: 0.867378
[8]	valid_0's multi_logloss: 0.854724
[9]	valid_0's multi_logloss: 0.840962
[10]	valid_0's multi_logloss: 0.826494
[11]	valid_0's multi_logloss: 0.814282
[12]	valid_0's multi_logloss: 0.800623
[13]	valid_0's multi_logloss: 0.788637
[14]	valid_0's multi_logloss: 0.778608
[15]	valid_0's multi_logloss: 0.765782
[16]	valid_0's multi_logloss: 0.754837
[17]	valid_0's multi_logloss: 0.742676
[18]	valid_0's multi_logloss: 0.732363
[19]	valid_0's multi_logloss: 0.720532
[20]	valid_0's multi_logloss: 0.711687
[21]	valid_0's multi_logloss: 0.702787
[22]	valid_0's multi_logloss: 0.694037
[23]	valid_0's multi_logloss: 0.683372
[24]	valid_0's multi_loglo

[206]	valid_0's multi_logloss: 0.211829
[207]	valid_0's multi_logloss: 0.211261
[208]	valid_0's multi_logloss: 0.210595
[209]	valid_0's multi_logloss: 0.209983
[210]	valid_0's multi_logloss: 0.209291
[211]	valid_0's multi_logloss: 0.208845
[212]	valid_0's multi_logloss: 0.208343
[213]	valid_0's multi_logloss: 0.207846
[214]	valid_0's multi_logloss: 0.207424
[215]	valid_0's multi_logloss: 0.206992
[216]	valid_0's multi_logloss: 0.206546
[217]	valid_0's multi_logloss: 0.206054
[218]	valid_0's multi_logloss: 0.205547
[219]	valid_0's multi_logloss: 0.205242
[220]	valid_0's multi_logloss: 0.205002
[221]	valid_0's multi_logloss: 0.204782
[222]	valid_0's multi_logloss: 0.204218
[223]	valid_0's multi_logloss: 0.203778
[224]	valid_0's multi_logloss: 0.203508
[225]	valid_0's multi_logloss: 0.203124
[226]	valid_0's multi_logloss: 0.202679
[227]	valid_0's multi_logloss: 0.202247
[228]	valid_0's multi_logloss: 0.201735
[229]	valid_0's multi_logloss: 0.201441
[230]	valid_0's multi_logloss: 0.201166


[414]	valid_0's multi_logloss: 0.168559
[415]	valid_0's multi_logloss: 0.168479
[416]	valid_0's multi_logloss: 0.16842
[417]	valid_0's multi_logloss: 0.168378
[418]	valid_0's multi_logloss: 0.16831
[419]	valid_0's multi_logloss: 0.168266
[420]	valid_0's multi_logloss: 0.168176
[421]	valid_0's multi_logloss: 0.168139
[422]	valid_0's multi_logloss: 0.16807
[423]	valid_0's multi_logloss: 0.167982
[424]	valid_0's multi_logloss: 0.167952
[425]	valid_0's multi_logloss: 0.167902
[426]	valid_0's multi_logloss: 0.167838
[427]	valid_0's multi_logloss: 0.167786
[428]	valid_0's multi_logloss: 0.167733
[429]	valid_0's multi_logloss: 0.167665
[430]	valid_0's multi_logloss: 0.167607
[431]	valid_0's multi_logloss: 0.167577
[432]	valid_0's multi_logloss: 0.167506
[433]	valid_0's multi_logloss: 0.16747
[434]	valid_0's multi_logloss: 0.167391
[435]	valid_0's multi_logloss: 0.16735
[436]	valid_0's multi_logloss: 0.167292
[437]	valid_0's multi_logloss: 0.167208
[438]	valid_0's multi_logloss: 0.167181
[439]

[622]	valid_0's multi_logloss: 0.16164
[623]	valid_0's multi_logloss: 0.161628
[624]	valid_0's multi_logloss: 0.161617
[625]	valid_0's multi_logloss: 0.1616
[626]	valid_0's multi_logloss: 0.161582
[627]	valid_0's multi_logloss: 0.161564
[628]	valid_0's multi_logloss: 0.16155
[629]	valid_0's multi_logloss: 0.161527
[630]	valid_0's multi_logloss: 0.161516
[631]	valid_0's multi_logloss: 0.161508
[632]	valid_0's multi_logloss: 0.161492
[633]	valid_0's multi_logloss: 0.161477
[634]	valid_0's multi_logloss: 0.161466
[635]	valid_0's multi_logloss: 0.161444
[636]	valid_0's multi_logloss: 0.161423
[637]	valid_0's multi_logloss: 0.161409
[638]	valid_0's multi_logloss: 0.1614
[639]	valid_0's multi_logloss: 0.161386
[640]	valid_0's multi_logloss: 0.161375
[641]	valid_0's multi_logloss: 0.161357
[642]	valid_0's multi_logloss: 0.161341
[643]	valid_0's multi_logloss: 0.161332
[644]	valid_0's multi_logloss: 0.161311
[645]	valid_0's multi_logloss: 0.161298
[646]	valid_0's multi_logloss: 0.161278
[647]	

[830]	valid_0's multi_logloss: 0.159864
[831]	valid_0's multi_logloss: 0.159856
[832]	valid_0's multi_logloss: 0.159852
[833]	valid_0's multi_logloss: 0.159849
[834]	valid_0's multi_logloss: 0.159849
[835]	valid_0's multi_logloss: 0.159844
[836]	valid_0's multi_logloss: 0.159839
[837]	valid_0's multi_logloss: 0.15983
[838]	valid_0's multi_logloss: 0.15982
[839]	valid_0's multi_logloss: 0.159805
[840]	valid_0's multi_logloss: 0.159804
[841]	valid_0's multi_logloss: 0.159804
[842]	valid_0's multi_logloss: 0.159806
[843]	valid_0's multi_logloss: 0.159802
[844]	valid_0's multi_logloss: 0.159798
[845]	valid_0's multi_logloss: 0.159801
[846]	valid_0's multi_logloss: 0.1598
[847]	valid_0's multi_logloss: 0.159795
[848]	valid_0's multi_logloss: 0.159786
[849]	valid_0's multi_logloss: 0.159778
[850]	valid_0's multi_logloss: 0.159776
[851]	valid_0's multi_logloss: 0.159772
[852]	valid_0's multi_logloss: 0.159769
[853]	valid_0's multi_logloss: 0.159765
[854]	valid_0's multi_logloss: 0.159759
[855

...

[69]	valid_0's multi_logloss: 0.409148
[70]	valid_0's multi_logloss: 0.406417
[71]	valid_0's multi_logloss: 0.402622
[72]	valid_0's multi_logloss: 0.399973
[73]	valid_0's multi_logloss: 0.396266
[74]	valid_0's multi_logloss: 0.393622
[75]	valid_0's multi_logloss: 0.390369
[76]	valid_0's multi_logloss: 0.387633
[77]	valid_0's multi_logloss: 0.384722
[78]	valid_0's multi_logloss: 0.381107
[79]	valid_0's multi_logloss: 0.379047
[80]	valid_0's multi_logloss: 0.375315
[81]	valid_0's multi_logloss: 0.372918
[82]	valid_0's multi_logloss: 0.369822
[83]	valid_0's multi_logloss: 0.367592
[84]	valid_0's multi_logloss: 0.364525
[85]	valid_0's multi_logloss: 0.361268
[86]	valid_0's multi_logloss: 0.35848
[87]	valid_0's multi_logloss: 0.355627
[88]	valid_0's multi_logloss: 0.353213
[89]	valid_0's multi_logloss: 0.35043
[90]	valid_0's multi_logloss: 0.347752
[91]	valid_0's multi_logloss: 0.345073
[92]	valid_0's multi_logloss: 0.342224
[93]	valid_0's multi_logloss: 0.340471
[94]	valid_0's multi_loglos

[276]	valid_0's multi_logloss: 0.18701
[277]	valid_0's multi_logloss: 0.186818
[278]	valid_0's multi_logloss: 0.18659
[279]	valid_0's multi_logloss: 0.186343
[280]	valid_0's multi_logloss: 0.186143
[281]	valid_0's multi_logloss: 0.185891
[282]	valid_0's multi_logloss: 0.185709
[283]	valid_0's multi_logloss: 0.185515
[284]	valid_0's multi_logloss: 0.185243
[285]	valid_0's multi_logloss: 0.184924
[286]	valid_0's multi_logloss: 0.184636
[287]	valid_0's multi_logloss: 0.184491
[288]	valid_0's multi_logloss: 0.184226
[289]	valid_0's multi_logloss: 0.183911
[290]	valid_0's multi_logloss: 0.183733
[291]	valid_0's multi_logloss: 0.183497
[292]	valid_0's multi_logloss: 0.183241
[293]	valid_0's multi_logloss: 0.182983
[294]	valid_0's multi_logloss: 0.182785
[295]	valid_0's multi_logloss: 0.182493
[296]	valid_0's multi_logloss: 0.182331
[297]	valid_0's multi_logloss: 0.18223
[298]	valid_0's multi_logloss: 0.181961
[299]	valid_0's multi_logloss: 0.181686
[300]	valid_0's multi_logloss: 0.181587
[30

[483]	valid_0's multi_logloss: 0.16492
[484]	valid_0's multi_logloss: 0.164881
[485]	valid_0's multi_logloss: 0.164863
[486]	valid_0's multi_logloss: 0.164835
[487]	valid_0's multi_logloss: 0.164792
[488]	valid_0's multi_logloss: 0.164754
[489]	valid_0's multi_logloss: 0.164732
[490]	valid_0's multi_logloss: 0.164705
[491]	valid_0's multi_logloss: 0.16466
[492]	valid_0's multi_logloss: 0.164639
[493]	valid_0's multi_logloss: 0.16458
[494]	valid_0's multi_logloss: 0.164536
[495]	valid_0's multi_logloss: 0.164471
[496]	valid_0's multi_logloss: 0.164431
[497]	valid_0's multi_logloss: 0.164409
[498]	valid_0's multi_logloss: 0.16437
[499]	valid_0's multi_logloss: 0.16433
[500]	valid_0's multi_logloss: 0.164308
[501]	valid_0's multi_logloss: 0.164277
[502]	valid_0's multi_logloss: 0.164239
[503]	valid_0's multi_logloss: 0.164185
[504]	valid_0's multi_logloss: 0.164151
[505]	valid_0's multi_logloss: 0.164127
[506]	valid_0's multi_logloss: 0.164104
[507]	valid_0's multi_logloss: 0.164071
[508]

[692]	valid_0's multi_logloss: 0.160645
[693]	valid_0's multi_logloss: 0.160643
[694]	valid_0's multi_logloss: 0.160627
[695]	valid_0's multi_logloss: 0.16062
[696]	valid_0's multi_logloss: 0.16062
[697]	valid_0's multi_logloss: 0.16061
[698]	valid_0's multi_logloss: 0.1606
[699]	valid_0's multi_logloss: 0.160591
[700]	valid_0's multi_logloss: 0.160576
[701]	valid_0's multi_logloss: 0.160564
[702]	valid_0's multi_logloss: 0.16055
[703]	valid_0's multi_logloss: 0.160527
[704]	valid_0's multi_logloss: 0.160516
[705]	valid_0's multi_logloss: 0.160497
[706]	valid_0's multi_logloss: 0.160484
[707]	valid_0's multi_logloss: 0.160479
[708]	valid_0's multi_logloss: 0.16047
[709]	valid_0's multi_logloss: 0.160454
[710]	valid_0's multi_logloss: 0.160448
[711]	valid_0's multi_logloss: 0.160442
[712]	valid_0's multi_logloss: 0.160439
[713]	valid_0's multi_logloss: 0.160431
[714]	valid_0's multi_logloss: 0.160421
[715]	valid_0's multi_logloss: 0.160413
[716]	valid_0's multi_logloss: 0.160402
[717]	v

[897]	valid_0's multi_logloss: 0.159486
[898]	valid_0's multi_logloss: 0.159488
[899]	valid_0's multi_logloss: 0.159487
[900]	valid_0's multi_logloss: 0.159487
[901]	valid_0's multi_logloss: 0.15948
[902]	valid_0's multi_logloss: 0.159477
[903]	valid_0's multi_logloss: 0.159475
[904]	valid_0's multi_logloss: 0.159473
[905]	valid_0's multi_logloss: 0.159472
[906]	valid_0's multi_logloss: 0.159465
[907]	valid_0's multi_logloss: 0.159465
[908]	valid_0's multi_logloss: 0.159464
[909]	valid_0's multi_logloss: 0.159458
[910]	valid_0's multi_logloss: 0.15946
[911]	valid_0's multi_logloss: 0.159465
[912]	valid_0's multi_logloss: 0.159456
[913]	valid_0's multi_logloss: 0.159453
[914]	valid_0's multi_logloss: 0.159451
[915]	valid_0's multi_logloss: 0.159452
[916]	valid_0's multi_logloss: 0.159449
[917]	valid_0's multi_logloss: 0.159449
[918]	valid_0's multi_logloss: 0.15945
[919]	valid_0's multi_logloss: 0.159449
[920]	valid_0's multi_logloss: 0.159449
[921]	valid_0's multi_logloss: 0.159443
[92

In [40]:
print(f'{accuracy_score(y, np.argmax(p_val, axis=1)) * 100:.4f}%')

93.2675%


In [41]:
#np.savetxt(p_val_file, p_val, fmt='%.6f', delimiter=',')
#np.savetxt(p_tst_file, p_tst, fmt='%.6f', delimiter=',')

피처 중요도 시각화


In [ ]:
imp = pd.DataFrame({'feature': df.columns, 'importance': clf.feature_importances_})
imp = imp.sort_values('importance').set_index('feature')
#imp.plot(kind='barh')

In [18]:

sub = pd.read_csv(sample_file, index_col=0)
#print(sub.shape)


(80000, 1)


In [19]:

sub[target_col] = np.argmax(p_tst, axis=1)
#sub.head()

,class
id,
320000,2
320001,0
320002,2
320003,0
320004,2


In [42]:
sub.to_csv(sub_file)


In [ ]:
#93.3216%
#93.3150%2000 n_es
#93.2675%2022 seed